# Artist Identification using Convolutional Neural Networks

In [ ]:

import numpy as np
import os
import matplotlib.pyplot as plt
import random
from PIL import Image

import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from numpy.random import seed
seed(1)


In [ ]:

path = '../input/images/'


# Dataset

In [ ]:

# Clean dataset -rename files with weird characters

# images = os.listdir(path + "Albrecht_Dürer/")

# for idx,img in enumerate(images):
#     img_path = path + img
#     new_img_name = path + "Albrecht_Durer_" + str(idx) + ".jpg"
#     os.rename(img_path, new_img_name)


## Data Distribution

In [ ]:

artist_frequency = os.listdir(path)

for (index,artist_folder) in enumerate(os.listdir(path)):
    artist_frequency[index] = (artist_frequency[index], len( [painting for painting in  os.listdir(path + "/" + artist_folder)] ) )


In [ ]:

x_pos = range(len(artist_frequency))

plt.figure(figsize=(15,15))
plt.barh(x_pos, [ artist[1] for artist in artist_frequency] )
plt.title("Frequency of Paintigs Per Artist")

plt.yticks(x_pos, [ artist[0].replace("_", " ") for artist in artist_frequency ])

plt.show()


In [ ]:

# Sort artists by number of paintings
artist_frequency.sort(key=lambda elem: -elem[1])

# Create a dataframe with artists having more than 200 paintings     ## Alternative method: choose maximum 100 pictures of each artist
artists_info = [ artist for artist in artist_frequency if artist[1] >= 200 ]
nr_total_paintings = sum( nr for (artist, nr) in artists_info)
artists_info = [ (artist,nr, nr_total_paintings/(nr*len(artists_info))) for (artist,nr) in artists_info ]

artist_classes = [ artist[0] for artist in artists_info ]
weights = [ artist[2] for artist in artists_info ]

for (idx, artist) in enumerate(artists_info):
    print(idx, artist[0].replace("_", " "), artist[1], round(artist[2], 4))


In [ ]:

x_pos = range(len(artists_info))

plt.figure(figsize=(15,15))
plt.barh(x_pos, [ artist[1] for artist in artists_info] )
plt.title("Frequency of Paintigs Per Artist")

plt.yticks(x_pos, [ artist.replace("_", " ") for artist in artist_classes])

plt.show()


In [ ]:

min_w = 2048
max_w = 0
min_h = 2048
max_h = 0

images = []

for artist_folder in os.listdir(path):
    if artist_folder in artist_classes:
        for painting in os.listdir(path + artist_folder):
            picture_path = path + artist_folder + "/" + painting 
            im = Image.open(picture_path)
            images.append(im) # label ish, tho se poate extrage si din im.filename

            h, w = im.size
            min_h = min(h, min_h)
            max_h = max(h, max_h)
            min_w = min(w, min_w)
            max_w = max(w, max_w)

print("Picture's dimensions are between:")
print("Height: ", min_h, max_h)
print("Width: ", min_w, max_w)


## Random samples from dataset

In [ ]:

def get_label(image): # image: PIL Image
    return image.filename.split('/')[3].replace("_", " ")


In [ ]:

# Print few random paintings
n = 5
fig, axes = plt.subplots(1, n, figsize=(20,10))

for i in range(n):
    random_artist = random.choice(artist_classes)
    random_image = random.choice(images)
    image = np.asarray(random_image)
    axes[i].imshow(image)
    axes[i].set_title("Pictor: " + random_artist.replace('_', ' '))
    axes[i].axis('off') # hide x, y axis

plt.show()


# Data Augmentation

## Custom Function

In [ ]:

def random_crop(x, crop_size=(224,224)):
    h, w, _ = x.shape
    range_w = (w - crop_size[1])
    range_h = (h - crop_size[0])
    offset_w = 0 if range_w == 0 else np.random.randint(range_w)
    offset_h = 0 if range_h == 0 else np.random.randint(range_h)
    cropped_x = x[offset_h:offset_h + crop_size[0], offset_w:offset_w + crop_size[1], :]
    return cropped_x


In [ ]:

def center_crop(x, crop_size=(224,224)):
    h, w, _ = x.shape
    center_h = h // 2
    center_w = w // 2
    offset_w = center_h - (crop_size[0] // 2)
    offset_h = center_h - (crop_size[0] // 2)
    cropped_x = x[offset_h:offset_h + crop_size[0], offset_w:offset_w + crop_size[1], :]
    return cropped_x


In [ ]:

def preprocessor(image):
    # perform augmentations here
    rotate = random.choice([0,1])
    if rotate == 0:
        aug = random_crop(image)
    else:
        angle = random.choice( [10*x for x in range(1,36)] )
        image = Image.fromarray(image)
        img = np.asarray(image.rotate(angle))
        aug = center_crop(img)
    return aug


In [ ]:

# Print a random painting and its customized random augmented version
fig, axes = plt.subplots(1, 2, figsize=(20,10))

random_artist = random.choice(artist_classes)
random_image = random.choice(images)

# Original image
image = np.asarray(random_image)
axes[0].imshow(image)
axes[0].set_title("An original Image of " + random_artist.replace('_', ' '))
axes[0].axis('off')

# Transformed image
aug_image = preprocessor(np.asarray(random_image))
axes[1].imshow(aug_image)
axes[1].set_title("A transformed Image of " + random_artist.replace('_', ' '))
axes[1].axis('off')

plt.show()


## ImageDataGenerator

In [ ]:

# Augment data
batch_size = 16
train_input_shape = (224, 224, 3)
n_classes = len(artist_classes)

train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255., # target values between 0 and 1 by scaling with a 1/255 - recommended
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
#                                    preprocessing_function=preprocessor
                                  )

train_generator = train_datagen.flow_from_directory(directory=path,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                    classes=artist_classes
                                                   )

valid_generator = train_datagen.flow_from_directory(directory=path,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                    classes=artist_classes
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID) # Train for 215 steps, validate for 53 steps


## Transformed pictures

In [ ]:

# Print a random painting and its random augmented version
fig, axes = plt.subplots(1, 2, figsize=(20,10))

random_artist = random.choice(artist_classes)
random_image = random.choice(os.listdir(os.path.join(path, random_artist)))
random_image_file = os.path.join(path, random_artist, random_image)

# Original image
image = plt.imread(random_image_file)
axes[0].imshow(image)
axes[0].set_title("An original image of " + random_artist.replace('_', ' '))
axes[0].axis('off')

# Transformed image
aug_image = train_datagen.random_transform(image)
axes[1].imshow(aug_image)
axes[1].set_title("A transformed image of " + random_artist.replace('_', ' '))
axes[1].axis('off')

plt.show()


# THE Convulsional Neural Network

## ResNet50

In [ ]:

# Load pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=train_input_shape)

for layer in base_model.layers:
    layer.trainable = True
    

## Layers

In [ ]:

# Add layers at the end
X = base_model.output
X = Flatten()(X)

X = Dense(512, kernel_initializer='he_uniform')(X) # params: units:int pozitiv, dimensiunea outputului | kernel_initializer - intitializatorul pt kernel (weights matrix) - distributie uniforma intr un interval pe baza de formule
#X = Dropout(0.5)(X) # takes in a float between 0 and 1, which is the fraction of the neurons to drop # helps prevent overfitting
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(16, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

output = Dense(n_classes, activation='softmax')(X)

model = Model(inputs=base_model.input, outputs=output)


## Optimizer

In [ ]:

optimizer = Adam(lr=0.0001) # stochastic gradient descent method
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])


## Callbacks

In [ ]:

n_epoch = 10

early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, # if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced
                              verbose=1, mode='auto')


In [ ]:

# Train the model - all layers
history1 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr],
#                               use_multiprocessing=True,
                              workers=16,
                              class_weight=weights
                             )


During transfer learning the first layers of the network are frozen while leaving the end layers open to modification


In [ ]:

# Freeze core ResNet layers and train again 
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[:50]:
    layer.trainable = True

optimizer = Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

n_epoch = 50
history2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr, early_stop],
#                               use_multiprocessing=True,
                              workers=16,
                              class_weight=weights
                             )


# Conclusions

## Accuracy & Loss

In [ ]:

# Loss value implies how poorly or well a model behaves after each iteration of optimization.

# Merge history1 and history2
history = {}
# history['loss'] = history1.history['loss'] + history2.history['loss']
# history['acc'] = history1.history['acc'] + history2.history['acc']
# history['val_loss'] = history1.history['val_loss'] + history2.history['val_loss']
# history['val_acc'] = history1.history['val_acc'] + history2.history['val_acc']
# history['lr'] = history1.history['lr'] + history2.history['lr']

# Fara a doua antrenare
history['loss'] = history1.history['loss']
history['acc'] = history1.history['accuracy']
history['val_loss'] = history1.history['val_loss']
history['val_acc'] = history1.history['val_accuracy']
history['lr'] = history1.history['lr']


In [ ]:

# Plot the training graph
def plot_training(history):
    acc = history['acc']
    val_acc = history['val_acc']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))

    fig, axes = plt.subplots(1, 2, figsize=(15,5))
    
    axes[0].plot(epochs, acc, 'r-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'b--', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].legend(loc='best')

    axes[1].plot(epochs, loss, 'r-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'b--', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].legend(loc='best')
    
    plt.show()
    
plot_training(history)


In [ ]:

# Prediction accuracy on train data
score = model.evaluate_generator(train_generator, verbose=1)
print("Prediction accuracy on train data =", score[1])


In [ ]:

# Prediction accuracy on CV data
score = model.evaluate_generator(valid_generator, verbose=1)
print("Prediction accuracy on validation data =", score[1])


## Confusion Matrix & Classification Report

In [ ]:

from sklearn.metrics import *
import seaborn as sns

tick_labels = artist_classes

def showClassficationReport_Generator(model, valid_generator, STEP_SIZE_VALID):
    # Loop on each generator batch and predict
    y_pred, y_true = [], []
    for i in range(STEP_SIZE_VALID):
        (X,y) = next(valid_generator)
        y_pred.append(model.predict(X))
        y_true.append(y)
    
    # Create a flat list for y_true and y_pred
    y_pred = [subresult for result in y_pred for subresult in result]
    y_true = [subresult for result in y_true for subresult in result]
    
    # Update Truth vector based on argmax
    y_true = np.argmax(y_true, axis=1)
    y_true = np.asarray(y_true).ravel()
    
    # Update Prediction vector based on argmax
    y_pred = np.argmax(y_pred, axis=1)
    y_pred = np.asarray(y_pred).ravel()
    
    # Confusion Matrix
    fig, ax = plt.subplots(figsize=(10,10))
    conf_matrix = confusion_matrix(y_true, y_pred, labels=np.arange(n_classes))
    conf_matrix = conf_matrix/np.sum(conf_matrix, axis=1)
    sns.heatmap(conf_matrix, annot=True, fmt=".2f", square=True, cbar=False, 
                cmap=plt.cm.jet, xticklabels=tick_labels, yticklabels=tick_labels,
                ax=ax)
    ax.set_ylabel('Actual')
    ax.set_xlabel('Predicted')
    ax.set_title('Confusion Matrix')
    plt.show()
    
    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=np.arange(n_classes), target_names=artist_classes))

showClassficationReport_Generator(model, valid_generator, STEP_SIZE_VALID)


# Demo

In [ ]:

# Prediction
from keras.preprocessing import *

n = 5
fig, axes = plt.subplots(1, n, figsize=(25,10))

for i in range(n):
    random_artist = random.choice(artist_classes)
    random_image = random.choice(os.listdir(os.path.join(path, random_artist)))
    random_image_file = os.path.join(path, random_artist, random_image)

    # Original image
    test_image = image.load_img(random_image_file, target_size=(train_input_shape[0:2]))

    # Predict artist
    test_image = image.img_to_array(test_image)
    test_image /= 255.
    test_image = np.expand_dims(test_image, axis=0)

    prediction = model.predict(test_image)
    prediction_probability = np.amax(prediction)
    prediction_idx = np.argmax(prediction)

    labels = train_generator.class_indices
    labels = dict((v,k) for k,v in labels.items())

    title = "Actual artist = {}\nPredicted artist = {}\nPrediction probability = {:.2f} %" \
                .format(random_artist.replace('_', ' '), labels[prediction_idx].replace('_', ' '),
                        prediction_probability*100)

    # Print image
    axes[i].imshow(plt.imread(random_image_file))
    axes[i].set_title(title)
    axes[i].axis('off')

plt.show()
